In [1]:
from pathlib import Path
import json, re, time
import pandas as pd

REPORT_DIR = Path("/home/dataset-assist-0/annual-reports_output")
OUT_DIR = Path("/home/dataset-assist-0/eval_outputs_geval")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 断点续跑：已存在就跳过
OUT_JSONL = OUT_DIR / "geval_llama2_13b.jsonl"
OUT_CSV   = OUT_DIR / "geval_llama2_13b.csv"

report_files = sorted(REPORT_DIR.glob("*_short_report*.txt"))
print("reports:", len(report_files), "example:", report_files[0] if report_files else None)

def load_text(p: Path, max_chars=20000):
    t = p.read_text(encoding="utf-8", errors="ignore")
    t = re.sub(r"\n{3,}", "\n\n", t).strip()
    return t[:max_chars]

# 你可以按需改评分维度（G-Eval）
RUBRIC = """You are an expert buy-side analyst and research editor.
Evaluate the given equity research summary (the "Report") for professional readers.

Score each dimension from 1 to 5:
- Professionalism: finance terminology, tone, concision, no fluff.
- Structure: clear sections, logical flow, scannable bullets where helpful.
- Clarity: unambiguous statements, avoids repetition.
- Actionability: highlights key drivers, risks, and implications.
- Overall: holistic quality.

Return ONLY valid JSON with keys:
professionalism, structure, clarity, actionability, overall, rationale (1 short paragraph).
"""

def build_prompt(report_text: str) -> str:
    return f"""{RUBRIC}

Report:
\"\"\"\n{report_text}\n\"\"\"
"""

print(build_prompt("Example report...")[:400])


reports: 8 example: /home/dataset-assist-0/annual-reports_output/NVIDIA-2018-Annual-Report_short_report.txt
You are an expert buy-side analyst and research editor.
Evaluate the given equity research summary (the "Report") for professional readers.

Score each dimension from 1 to 5:
- Professionalism: finance terminology, tone, concision, no fluff.
- Structure: clear sections, logical flow, scannable bullets where helpful.
- Clarity: unambiguous statements, avoids repetition.
- Actionability: highlights 


In [2]:
import os, sys, subprocess, time, requests, textwrap, signal

MODEL_DIR = "/home/dataset-assist-0/data/models/llama2-13b"
VLLM_BASE = "http://127.0.0.1:8000/v1"

env = os.environ.copy()
env["TRANSFORMERS_OFFLINE"] = "1"
env["HF_HUB_OFFLINE"] = "1"

cmd = [
    sys.executable, "-m", "vllm.entrypoints.openai.api_server",
    "--model", MODEL_DIR,
    "--host", "127.0.0.1", "--port", "8000",
    "--dtype", "auto",
    "--tensor-parallel-size", "2",
    "--max-model-len", "4096",
    "--gpu-memory-utilization", "0.90",
]

proc = subprocess.Popen(cmd, env=env, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# 等待服务起来
ok = False
for _ in range(120):
    try:
        r = requests.get(f"{VLLM_BASE}/models", timeout=1.5)
        if r.status_code == 200:
            ok = True
            break
    except Exception:
        pass
    time.sleep(1)

if not ok:
    # 打印最近日志，方便定位
    lines = []
    try:
        for _ in range(80):
            lines.append(proc.stdout.readline())
    except Exception:
        pass
    raise RuntimeError("vLLM failed to start.\nLast logs:\n" + "".join(lines))

print("vLLM is up:", requests.get(f"{VLLM_BASE}/models", timeout=10).json())

RuntimeError: vLLM failed to start.
Last logs:
[0;36m(APIServer pid=241583)[0;0m INFO 12-29 20:02:24 [api_server.py:1351] vLLM API server version 0.13.0
[0;36m(APIServer pid=241583)[0;0m INFO 12-29 20:02:24 [utils.py:253] non-default args: {'host': '127.0.0.1', 'model': '/home/dataset-assist-0/data/models/llama2-13b', 'max_model_len': 4096, 'tensor_parallel_size': 2}
[0;36m(APIServer pid=241583)[0;0m INFO 12-29 20:02:24 [model.py:514] Resolved architecture: LlamaForCausalLM
[0;36m(APIServer pid=241583)[0;0m INFO 12-29 20:02:24 [model.py:1661] Using max model len 4096
[0;36m(APIServer pid=241583)[0;0m INFO 12-29 20:02:25 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=2048.
[0;36m(EngineCore_DP0 pid=241726)[0;0m INFO 12-29 20:02:31 [core.py:93] Initializing a V1 LLM engine (v0.13.0) with config: model='/home/dataset-assist-0/data/models/llama2-13b', speculative_config=None, tokenizer='/home/dataset-assist-0/data/models/llama2-13b', skip_tokenizer_in

In [ ]:
import requests, json
from tqdm.auto import tqdm

VLLM_BASE = "http://127.0.0.1:8000/v1"
MODEL_ID = "llama2-13b-judge"

def chat_once(prompt: str, temperature=0.0, max_tokens=400):
    payload = {
        "model": MODEL_ID,
        "messages": [
            {"role": "system", "content": "You are a strict evaluator. Output JSON only."},
            {"role": "user", "content": prompt},
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
    }
    r = requests.post(f"{VLLM_BASE}/chat/completions", json=payload, timeout=120)
    r.raise_for_status()
    return r.json()["choices"][0]["message"]["content"]

def extract_json(s: str):
    m = re.search(r"\{.*\}", s, flags=re.S)
    if not m:
        raise ValueError("No JSON found")
    return json.loads(m.group(0))

# 断点续跑：读已有 jsonl
done = set()
rows = []
if OUT_JSONL.exists():
    for line in OUT_JSONL.read_text(encoding="utf-8").splitlines():
        if not line.strip(): 
            continue
        obj = json.loads(line)
        done.add(obj["file"])
        rows.append(obj)

print("already done:", len(done))

with OUT_JSONL.open("a", encoding="utf-8") as f:
    for rp in tqdm(report_files):
        if rp.name in done:
            continue

        report_text = load_text(rp, max_chars=20000)
        prompt = build_prompt(report_text)

        t0 = time.time()
        try:
            raw = chat_once(prompt, temperature=0.0, max_tokens=400)
            j = extract_json(raw)
            rec = {
                "file": rp.name,
                "professionalism": j.get("professionalism"),
                "structure": j.get("structure"),
                "clarity": j.get("clarity"),
                "actionability": j.get("actionability"),
                "overall": j.get("overall"),
                "rationale": j.get("rationale"),
                "latency_s": round(time.time() - t0, 3),
            }
        except Exception as e:
            rec = {
                "file": rp.name,
                "error": str(e),
                "latency_s": round(time.time() - t0, 3),
            }

        f.write(json.dumps(rec, ensure_ascii=False) + "\n")
        f.flush()
        rows.append(rec)
        done.add(rp.name)

df = pd.DataFrame(rows)
df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
df.head(), df.shape
